In [1]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

from tensorflow.keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau, CSVLogger
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

from Codes.custom_datagen import imageLoader,load_img
import random

import segmentation_models_3D as sm

2.8.0
2.8.0
Segmentation Models: using `tf.keras` framework.


In [2]:
# Define the Dice coefficient function
def dice_coef(y_true, y_pred):
    smooth = 1e-15
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

# Define the Dice coefficient loss function
def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)  # Use 1 - Dice coefficient as the loss


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

def dice_coef(y_true, y_pred):
    smooth = 1e-15
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def iou(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + 1e-15) / (union + 1e-15)

# def dice_coef_loss(y_true, y_pred):
#     return -dice_coef(y_true, y_pred)

In [11]:
from keras.models import load_model
model = load_model('training/SP/brats_3d_full_sp.hdf5', custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef, 'iou':iou})

In [17]:
val_img_dir = "D:/1803070/Brats/Splited_Combined/val/images/"
val_mask_dir = "D:/1803070/Brats/Splited_Combined/val/masks/"

test_img_dir = "D:/1803070/Brats/Splited_Combined/test/images/"
test_mask_dir = "D:/1803070/Brats/Splited_Combined/test/masks/"

val_img_list=os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

test_img_list=os.listdir(test_img_dir)
test_mask_list = os.listdir(test_mask_dir)

batch_size = 2

val_img_datagen = imageLoader(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

test_img_datagen = imageLoader(test_img_dir, test_img_list, 
                                test_mask_dir, test_mask_list, batch_size)

In [19]:
def calculate_metrics(model, data_generator, steps):
    dice_scores = []
    iou_scores = []
    
    for _ in range(steps):
        X_val, Y_val = next(data_generator)
        predictions = model.predict(X_val)
        
#         # Calculate dice score and IoU for each sample in the batch
#         for i in range(len(X_val)):
#             dice = dice_coef(Y_val[i], predictions[i])
#             i = iou(Y_val[i], predictions[i])
#             dice_scores.append(dice)
#             iou_scores.append(i)


        # Calculate dice score and IoU for whole batch
        dice = dice_coef(Y_val, predictions)
        i = iou(Y_val, predictions)
        dice_scores.append(dice)
        iou_scores.append(i)
    
    # Calculate average scores
    avg_dice = sum(dice_scores) / len(dice_scores)
    avg_iou = sum(iou_scores) / len(iou_scores)
    
    return avg_dice, avg_iou

# Assuming you want to evaluate on a certain number of steps/batches
num_val_steps = 10 

# Calculate metrics on validation data
avg_val_dice, avg_val_iou = calculate_metrics(model, val_img_datagen, num_val_steps)

print(f'Average Dice Score on Validation Data: {avg_val_dice}')
print(f'Average IoU on Validation Data: {avg_val_iou}')


# Calculate metrics on validation data
avg_test_dice, avg_test_iou = calculate_metrics(model, test_img_datagen, num_val_steps)

print(f'\nAverage Dice Score on Test Data: {avg_test_dice}')
print(f'Average IoU on Test Data: {avg_test_iou}')

Average Dice Score on Validation Data: 0.9932529330253601
Average IoU on Validation Data: 0.9866235852241516

Average Dice Score on Test Data: 0.9896089434623718
Average IoU on Test Data: 0.9796287417411804


In [20]:
import numpy as np
import tensorflow as tf

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


def calculate_metrics_per_class(model, data_generator, steps, num_classes):
    dice_scores_per_class = {i: [] for i in range(num_classes)}
    iou_scores_per_class = {i: [] for i in range(num_classes)}
    
    for _ in range(steps):
        X_val, Y_val = next(data_generator)
        predictions = model.predict(X_val)
        
        # Calculate dice score and IoU for each class in each sample
#         for i in range(len(X_val)):
#             p = Y_val[i]
#             q = predictions[i]
        for class_idx in range(num_classes):
            y_true_class = Y_val[:, :, :, :, class_idx]
            y_pred_class = predictions[:, :, :, :, class_idx]

            dice = dice_coef(y_true_class, y_pred_class)
            i = iou(y_true_class, y_pred_class)

            dice_scores_per_class[class_idx].append(dice)
            iou_scores_per_class[class_idx].append(i)
    
    # Calculate average scores for each class
    avg_dice_per_class = {i: sum(scores) / len(scores) for i, scores in dice_scores_per_class.items()}
    avg_iou_per_class = {i: sum(scores) / len(scores) for i, scores in iou_scores_per_class.items()}
    
    return avg_dice_per_class, avg_iou_per_class

# Assuming you want to evaluate on a certain number of steps/batches
num_steps = 10  # Change this to suit your testing preference
num_classes = 4  # Change this to the number of classes

In [21]:
# Calculate metrics on validation data for each class
avg_val_dice_per_class, avg_val_iou_per_class = calculate_metrics_per_class(model, val_img_datagen, num_steps, num_classes)

# Print the results for each class
for class_idx in range(num_classes):
    avg_dice = avg_val_dice_per_class[class_idx]
    avg_iou = avg_val_iou_per_class[class_idx]
    print(f'Class {class_idx} - Average Dice Score on Validation Data: {avg_dice}')
    print(f'Class {class_idx} - Average IoU on Validation Data: {avg_iou}')

# Calculate mean across all classes
mean_dice = np.mean(list(avg_val_dice_per_class.values()))
mean_iou = np.mean(list(avg_val_iou_per_class.values()))

print(f"Mean Dice Score on Validation Data: {mean_dice}")
print(f"Mean IoU on Validation Data: {mean_iou}")

Class 0 - Average Dice Score on Validation Data: 0.9968904256820679
Class 0 - Average IoU on Validation Data: 0.9938055872917175
Class 1 - Average Dice Score on Validation Data: 0.7460901737213135
Class 1 - Average IoU on Validation Data: 0.6343157887458801
Class 2 - Average Dice Score on Validation Data: 0.7934149503707886
Class 2 - Average IoU on Validation Data: 0.6796221733093262
Class 3 - Average Dice Score on Validation Data: 0.8403255343437195
Class 3 - Average IoU on Validation Data: 0.7266498804092407
Mean Dice Score on Validation Data: 0.8441802859306335
Mean IoU on Validation Data: 0.7585983276367188


In [22]:
# Calculate metrics on validation data for each class
avg_test_dice_per_class, avg_test_iou_per_class = calculate_metrics_per_class(model, test_img_datagen, num_steps, num_classes)



# Print the results for each class
for class_idx in range(num_classes):
    avg_dice = avg_test_dice_per_class[class_idx]
    avg_iou = avg_test_iou_per_class[class_idx]
    print(f'Class {class_idx} - Average Dice Score on Test Data: {avg_dice}')
    print(f'Class {class_idx} - Average IoU on Test Data: {avg_iou}')

# Calculate mean across all classes
mean_dice = np.mean(list(avg_test_dice_per_class.values()))
mean_iou = np.mean(list(avg_test_iou_per_class.values()))

print(f"Mean Dice Score on Test Data: {mean_dice}")
print(f"Mean IoU on Test Data: {mean_iou}")


Class 0 - Average Dice Score on Test Data: 0.9974530935287476
Class 0 - Average IoU on Test Data: 0.9949232339859009
Class 1 - Average Dice Score on Test Data: 0.6932392716407776
Class 1 - Average IoU on Test Data: 0.5565176010131836
Class 2 - Average Dice Score on Test Data: 0.8495780229568481
Class 2 - Average IoU on Test Data: 0.7463930249214172
Class 3 - Average Dice Score on Test Data: 0.8167829513549805
Class 3 - Average IoU on Test Data: 0.6957644820213318
Mean Dice Score on Test Data: 0.8392633199691772
Mean IoU on Test Data: 0.7483996152877808
